In [1]:
import os
import math
import glob
from scipy.stats import multivariate_normal as mvn
import librosa as lr
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib
import librosa as lr
import IPython
import scipy.io as sio
import scipy

In [2]:

s =lr.load("trs.wav",sr = None)[0]
n=lr.load("trn.wav",sr=None)[0]
print(s)
print("S:",s.shape)
print("N:",n.shape)

[ 2.4414062e-04 -1.8310547e-04  1.2207031e-04 ... -3.0517578e-05
 -9.1552734e-05  3.0517578e-05]
S: (403255,)
N: (403255,)


In [3]:
IPython.display.Audio(s,rate = 16000)

In [4]:
IPython.display.Audio(n,rate = 16000)

In [5]:
X=s+n
print("X:",X.shape)

X: (403255,)


In [6]:
IPython.display.Audio(X,rate = 16000)

In [7]:
N_samples=1024
s_stft = lr.stft(s,n_fft =N_samples,hop_length=512,window="hann")
n_stft = lr.stft(n,n_fft =N_samples,hop_length=512,window="hann")
X_stft = lr.stft(X,n_fft =N_samples,hop_length=512,window="hann")

In [8]:
#discarding the complex conjugate
s_real = np.abs(s_stft)
n_real = np.abs(n_stft)
X_real = np.abs(X_stft)

#size of each spectrum
print("S_real: ",s_real.shape)
print("N_real: ",n_real.shape)
print("X_real: ",X_real.shape)

S_real:  (513, 788)
N_real:  (513, 788)
X_real:  (513, 788)


In [9]:
X_input = np.abs(X_real)

In [10]:
#binary mask
M = np.zeros(s_real.shape)
for col in range(s_real.shape[1]):
    for row in range(s_real.shape[0]):
        if np.abs(s_real[row][col])>np.abs(n_real[row][col]):
            M[row][col]=1

In [11]:
def layers_Weight_initialization():
    np.random.seed(1)
    W1 = np.random.randn(50,513)
    W2 = np.random.randn(513,50)
    
    b1 = np.random.randn(50,1)
    b2 = np.random.randn(513,1)
    
    return W1,W2,b1,b2

def relu_activation(input_units):
        # relu activation function is 0 for all values less than 0
        cost_ftn=np.maximum(0,input_units)
        return cost_ftn
    
def relu_prime(x):
        return np.max(np.sign(x),0)

def sigmoid(input_x):
    activationValue = 1/(1+np.exp(-input_x))
    return activationValue

def derivative_sigmoid(input_y):
    output = sigmoid(input_y)*(1-sigmoid(input_y))
    return output

In [12]:
def train(X_data,W1,W2,b1,b2,Y_actual,lr,iter_):
    errsq_list=[]
    acc_list=[]
    for i in range(iter_):
        #forward propagation
        Z1 = W1@X_data+b1
        inp2 = relu_activation(Z1)
    
        Z2 = W2@inp2+b2
        Y_pred = sigmoid(Z2)

        #backward propagation
        delta_L2 = (Y_pred-Y_actual) * derivative_sigmoid(Z2)
        dw_l2 = delta_L2@inp2.T
        delta_L1 = W2.T @ delta_L2 * relu_prime(Z1)
        dw_l1 = delta_L1@X_data.T

        #updating the weights and bias
        W1=W1-(lr*dw_l1)
        W2=W2-(lr*dw_l2)
        b1=b1-np.mean(delta_L1,axis=-1)[:,np.newaxis]
        b2=b2-np.mean(delta_L2,axis=-1)[:,np.newaxis]


        Y_classpred = Y_pred>0.5
        #calculate the accuracy
        acc=(np.sum(Y_actual==Y_classpred))/(Y_actual.shape[0]*Y_actual.shape[1])
        acc_list.append(acc)

        #calculate squared err
        err_sq= (1/2)*np.sum((Y_pred-Y_actual)**2)
        errsq_list.append(err_sq)
        
        if i%1000==0:
            print("iteration:",i,"---","accuracy:",acc,"Squared Error:",err_sq)

    return errsq_list,acc_list,W1,W2,b1,b2,Y_classpred.astype(int)

In [13]:
W1,W2,b1,b2=layers_Weight_initialization()
error_4,accuracy_4,W1_tr,W2_tr,b1_tr,b2_tr,y_pred = train(X_input,W1,W2,b1,b2,M,0.01,3250)

iteration: 0 --- accuracy: 0.5066964506585132 Squared Error: 95151.45314813948
iteration: 1000 --- accuracy: 0.7345860420933892 Squared Error: 53646.000002323904
iteration: 2000 --- accuracy: 0.7345860420933892 Squared Error: 53646.0000010726
iteration: 3000 --- accuracy: 0.7345860420933892 Squared Error: 53646.00000069474
